# Gather split statistics

Problem: Split statistics are held in individual txt-files. 
To make use of the more easily, it would be convenient to gather all this information into a single csv file.

In [12]:
import pathlib
import re
import sys
sys.path.append(str(pathlib.Path("__file__").absolute().parents[1]))

import pandas as pd

from src.util.definitions import DATA_ROOT

In [2]:
# read all the statistics files
rows = []
for split in (DATA_ROOT / "splits").iterdir():
    match_split = re.search(r"synferm_dataset_2023-12-20_(\dD)_split_*(.*)", split.name)
    match_split_synthetic = re.search(r"synferm_dataset_2024-01-31_synthetic_(\dD)_split_*(.*)", split.name)
    if split.is_dir():
        if match_split:
            split_name = "_".join(match_split.groups()).strip("_")
            for file in split.iterdir():
                match = re.search(r"fold(\d+)_statistics\.txt", file.name)
                if match:
                    with open(file, "r") as f:
                        content = {"split_name": split_name, "fold": match.group(1)}
                        content.update({k:v  for k,v in [line.strip("\n").split(": ") for line in f.readlines()]})
                        rows.append(content)                
        elif match_split_synthetic:
            split_name = "_".join(match_split_synthetic.groups()).strip("_")
            for file in split.iterdir():
                match = re.search(r"fold(\d+)_statistics\.txt", file.name)
                if match:
                    with open(file, "r") as f:
                        content = {"split_name": f"{split_name}_syn", "fold": match.group(1)}
                        content.update({k:v  for k,v in [line.strip("\n").split(": ") for line in f.readlines()]})
                        rows.append(content)  

In [3]:
# create df to hold all info
df = pd.DataFrame(rows).sort_values(by=["split_name", "fold"], ignore_index=True)
df

,split_name,fold,Train samples,Val samples,Test samples,Train samples binary_A has label 1,Train samples binary_B has label 1,Train samples binary_C has label 1,Val samples binary_A has label 1,Val samples binary_B has label 1,...,Train initiators,Val initiators,Test initiators,Train monomers,Val monomers,Test monomers,Train terminators,Val terminators,Test terminators,Not used
0,0D_0.625,0,246 (0.6%),19743 (50.0%),19497 (49.4%),195 (79.3%),123 (50.0%),67 (27.2%),16267 (82.4%),11439 (57.9%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0D_0.625,1,246 (0.6%),19743 (50.0%),19497 (49.4%),197 (80.1%),145 (58.9%),77 (31.3%),16227 (82.2%),11458 (58.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0D_0.625,2,246 (0.6%),19743 (50.0%),19497 (49.4%),203 (82.5%),148 (60.2%),78 (31.7%),16201 (82.1%),11430 (57.9%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0D_0.625,3,246 (0.6%),19743 (50.0%),19497 (49.4%),197 (80.1%),141 (57.3%),65 (26.4%),16214 (82.1%),11456 (58.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0D_0.625,4,246 (0.6%),19743 (50.0%),19497 (49.4%),204 (82.9%),120 (48.8%),56 (22.8%),16223 (82.2%),11448 (58.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,3D_80,4,19544 (49.5%),43 (0.1%),26 (0.1%),16373 (83.8%),11539 (59.0%),5475 (28.0%),29 (67.4%),17 (39.5%),...,53,7,4,48,7,8,32,4,4,19873 (50.3%)
357,3D_80,5,19795 (50.1%),38 (0.1%),39 (0.1%),16226 (82.0%),10824 (54.7%),5663 (28.6%),35 (92.1%),16 (42.1%),...,53,7,7,48,7,8,32,4,5,19614 (49.7%)
358,3D_80,6,18709 (47.4%),48 (0.1%),68 (0.2%),15472 (82.7%),10854 (58.0%),4864 (26.0%),44 (91.7%),37 (77.1%),...,53,7,7,47,6,9,32,4,5,20661 (52.3%)
359,3D_80,7,19819 (50.2%),49 (0.1%),46 (0.1%),15976 (80.6%),11722 (59.1%),6187 (31.2%),47 (95.9%),29 (59.2%),...,53,6,7,48,7,7,32,4,5,19572 (49.6%)


In [8]:
# save to file for later use
df.to_csv(DATA_ROOT / "splits" / "split_statistics_2023-12-20.csv", index=False)

In [13]:
df.loc[df["split_name"] == "2D"]

,split_name,fold,Train samples,Val samples,Test samples,Train samples binary_A has label 1,Train samples binary_B has label 1,Train samples binary_C has label 1,Val samples binary_A has label 1,Val samples binary_B has label 1,...,Train initiators,Val initiators,Test initiators,Train monomers,Val monomers,Test monomers,Train terminators,Val terminators,Test terminators,Not used
145,2D,0,13206 (33.4%),1738 (4.4%),1756 (4.4%),10818 (81.9%),7819 (59.2%),3570 (27.0%),1404 (80.8%),959 (55.2%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22786 (57.7%)
146,2D,1,13087 (33.1%),1922 (4.9%),1658 (4.2%),10668 (81.5%),7626 (58.3%),3429 (26.2%),1683 (87.6%),1166 (60.7%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22819 (57.8%)
147,2D,2,14890 (37.7%),1415 (3.6%),1615 (4.1%),12164 (81.7%),8560 (57.5%),4204 (28.2%),1218 (86.1%),877 (62.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21566 (54.6%)
148,2D,3,13375 (33.9%),1546 (3.9%),1891 (4.8%),11046 (82.6%),7905 (59.1%),3957 (29.6%),1322 (85.5%),1163 (75.2%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22674 (57.4%)
149,2D,4,14278 (36.2%),1574 (4.0%),1487 (3.8%),12408 (86.9%),9551 (66.9%),4882 (34.2%),1058 (67.2%),800 (50.8%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22147 (56.1%)
150,2D,5,13462 (34.1%),1461 (3.7%),1916 (4.9%),11314 (84.0%),7766 (57.7%),4157 (30.9%),932 (63.8%),583 (39.9%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22647 (57.4%)
151,2D,6,14138 (35.8%),1398 (3.5%),1957 (5.0%),11569 (81.8%),7415 (52.4%),3662 (25.9%),1182 (84.5%),825 (59.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21993 (55.7%)
152,2D,7,14268 (36.1%),1619 (4.1%),1624 (4.1%),11423 (80.1%),7291 (51.1%),3659 (25.6%),1426 (88.1%),1183 (73.1%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21975 (55.7%)
153,2D,8,12864 (32.6%),1791 (4.5%),1736 (4.4%),9914 (77.1%),5868 (45.6%),2664 (20.7%),1573 (87.8%),1305 (72.9%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23095 (58.5%)
